In [1]:
import random
import networkx as nx
import itertools

In [2]:
def load_gcol(file_path):
    G = nx.Graph() 

    with open(file_path, "r") as file:
        for line in file:
            parts = line.split()
            if parts[0] == 'e':
                node1 = int(parts[1]) 
                node2 = int(parts[2])  
                G.add_edge(node1, node2) 
    return G

In [3]:
def generate_population(size,G):
    return [[random.randint(0, 1) for _ in range(len(G.nodes))] for _ in range(size)]

In [5]:
def fitness(individual, G, shortest_paths):
    resolving_set = {i for i, bit in enumerate(individual) if bit == 1}

    if not resolving_set:  # Ako je skup prazan, penalizacija
        return float("inf")

    unresolved_pairs = 0
    for u, v in itertools.combinations(G.nodes, 2):
        if not any(abs(shortest_paths[u][j] - shortest_paths[v][j]) > 0 for j in resolving_set if j in shortest_paths[u]):
            unresolved_pairs += 1  # Par (u, v) nije razrešen

    return len(resolving_set) + 100 * unresolved_pairs  # Penalizacija nerešenih parova

In [6]:
def tournament_selection(population, G, k=3):
    selected = random.sample(population, k)
    return min(selected, key=lambda ind: fitness(ind, G))

In [7]:
def weighted_selection(population, G,shortest_paths):
    fitness_values = [1 / (1 + fitness(ind, G,shortest_paths)) for ind in population]  # Inverzija jer tražimo minimu
    return random.choices(population, weights=fitness_values, k=2)  # Bira 2 roditelja

In [8]:
def crossover(parent1, parent2,G ):
    point = random.randint(1, len(G.nodes) - 1)
    return parent1[:point] + parent2[point:], parent2[:point] + parent1[point:]

In [9]:
def mutate(individual, mutation_rate=0.05):
    return [bit if random.random() > mutation_rate else 1 - bit for bit in individual]

In [10]:
def genetic_algorithm(G,shortest_paths,pop_size=100, generations=200, crossover_rate=0.7, mutation_rate=0.1):
    population = generate_population(pop_size,G)
    
    for gen in range(generations):
        new_population = []
      
        for _ in range(pop_size // 2):
            # parent1 = tournament_selection(population,G)
            # parent2 = tournament_selection(population,G)
            parent1, parent2 = weighted_selection(population, G,shortest_paths)            
            # Ukrštanje
            if random.random() < crossover_rate:
                child1, child2 = crossover(parent1, parent2,G)
            else:
                child1, child2 = parent1, parent2
            
            # Mutacija
            child1 = mutate(child1, mutation_rate)
            child2 = mutate(child2, mutation_rate)

            new_population.extend([child1, child2])

        population = new_population
        
        # Pronalazak najbolje jedinke
        best_solution = min(population, key=lambda ind: fitness(ind, G,shortest_paths))
        best_fitness = fitness(best_solution,G,shortest_paths)

        print(f"Generacija {gen+1}, Najbolji fitnes: {best_fitness}")

       

    # Prikaz optimalnog rešenja
    optimal_set = {i for i, bit in enumerate(best_solution) if bit == 1}
    print("\nOptimalni rešavajući skup:", optimal_set)
    return optimal_set

In [11]:
# G = nx.petersen_graph()
G = load_gcol('grafovi\gcol\gcol1.txt')
# G = nx.Graph()
# G.add_edges_from([(1,2),(1,3),(1,5),(1,6),(2,4),(2,6),(3,5),(3,4),(4,5),(4,6),(5,6)])

shortest_paths = dict(nx.all_pairs_shortest_path_length(G))  # Svi putevi između svih parova


genetic_algorithm(G,shortest_paths)

<>:2: SyntaxWarning: invalid escape sequence '\g'
<>:2: SyntaxWarning: invalid escape sequence '\g'
C:\Users\pc\AppData\Local\Temp\ipykernel_14016\4233301167.py:2: SyntaxWarning: invalid escape sequence '\g'
  G = load_gcol('grafovi\gcol\gcol1.txt')


Generacija 1, Najbolji fitnes: 30
Generacija 2, Najbolji fitnes: 39
Generacija 3, Najbolji fitnes: 35


C:\Users\pc\AppData\Local\Temp\ipykernel_14016\4233301167.py:2: SyntaxWarning: invalid escape sequence '\g'
  G = load_gcol('grafovi\gcol\gcol1.txt')


KeyboardInterrupt: 